In [1]:
import pandas as pd
from pandas import HDFStore
from pathlib import Path
import warnings

# Speichert das ganze als hdf5 ab (jede Zeile als eigenes key-value pair mit der zur zeile gehörenden gid als key. also die distanzen von key_gid zu allen anderen, Reihenfolge wie im header.

In [10]:
%time
distanzmatrix_name = "Distanzmatrix.h5"#_table.h5"
MAIN_DIR = Path("/media/praisethesun/INTENSO/DistMatProj/DR/Distanzmatrix/")
ROWS_AS_SERIES = MAIN_DIR/"rows_as_series"

store = HDFStore(MAIN_DIR/distanzmatrix_name)

gids = pd.read_csv(MAIN_DIR/"header_gids.csv", header=None).squeeze()
store["header_gids"] = gids

c = 0
# HDF5 hat wohl ein Problem wenn die keys nur nummern sind,
# das gibt dann eine Warning die ich unterdrückt hab weil die
# Ausgabe sonst nervt
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for gid in gids:
        row = pd.read_csv(ROWS_AS_SERIES/f"{gid}.csv", header=None).squeeze()
        store.put(key=str(gid), value=row)# , format='table')
        c += 1
        if c % 1000 == 0:
           print(c)
store.close()

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 12.9 µs
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000


## Hier nochmal testweise beide versionen zum auswählen, unten nochmal die endversion

In [26]:
def get_distances(list_of_gids, version='fixed'):
    list_of_gids = pd.Series(list_of_gids)

    MAIN_DIR = Path("/media/praisethesun/INTENSO/DistMatProj/DR/Distanzmatrix/") # better define globally

    distanzmatrix_name = "Distanzmatrix_table.h5" if version == 'table' else "Distanzmatrix.h5"

    store = HDFStore(MAIN_DIR/distanzmatrix_name)
    header = store["header_gids"]

    gids_not_found = list_of_gids[~list_of_gids.isin(header)]
    if len(gids_not_found) > 0:
        raise ValueError("Following gids not found: " + gids_not_found.to_string())

    query_mask = gids.isin(list_of_gids)
    list_of_gids = gids[query_mask] # reorder

    series = []
    for gid in list_of_gids:
        if version == 'table':
            next = store.select(str(gid), where=query_mask)
        else:
            next = store[str(gid)]
            next = next[query_mask]
        series.append(next)

    res = pd.DataFrame(series)
    res.columns = list_of_gids
    res.index = list_of_gids

    store.close()
    return res


# TESTS etc.

In [22]:
DATA_DIR = Path("/media/praisethesun/INTENSO/DistMatProj/DR/Distanzmatrix/")
ROWS_DIR = DATA_DIR/ "as_rows"
AS_COLUMNS = DATA_DIR/"rows_as_series"

In [29]:
gids = pd.read_csv(DATA_DIR/"header_gids.csv", header=None).squeeze()
list_of_gids = gids.sample(5000)

In [30]:
%timeit get_distances(list_of_gids, version='fixed')

25.7 s ± 17.7 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit get_distances(list_of_gids, version='table')

1min 17s ± 21.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Deswegen hatte ich die Zeilen nochmal als Series/Spalte abgespeichert (ersteres sind die Zeilen, letzteres die Zeilen als "Spalten"):

In [ ]:
gid = gids[42]

In [113]:
%timeit row = pd.read_csv(ROWS_DIR/f"{gid}.csv", header=None)

1.18 s ± 39.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [109]:
%timeit row = pd.read_csv(AS_COLUMNS/f"{gid}.csv", header=None)

9.75 ms ± 889 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Hier die richtige get_distances():

In [2]:
def get_distances(list_of_gids):

    # list_of_gids = pd.Series(list_of_gids)
    # muss dann als pandas.Series übergeben werden

    MAIN_DIR = Path("/media/praisethesun/INTENSO/DistMatProj/DR/Distanzmatrix/") # better define globally

    store = HDFStore(MAIN_DIR/"Distanzmatrix.h5")
    header = store["header_gids"]

    # vielleicht nicht nötig wenn der Fall vorher abgefangen wird
    gids_not_found = list_of_gids[~list_of_gids.isin(header)]
    if len(gids_not_found) > 0:
        raise ValueError("Following gids not found: " + gids_not_found.to_string(index=False))

    query_mask = header.isin(list_of_gids)
    list_of_gids = header[query_mask] # reorder

    series = []
    for gid in list_of_gids:
        next = store[str(gid)]
        next = next[query_mask]
        series.append(next)

    res = pd.DataFrame(series)
    res.columns = list_of_gids
    res.index = list_of_gids

    store.close()
    return res


In [5]:
gids = [522647,522710, 1]
t = pd.Series(gids)
try:
    get_distances(t).to_json(orient='split')
except ValueError as val_err:
    print(val_err)

Following gids not found: 2    1


In [8]:
gids = [522647,522710, 1]
t = pd.Series(gids)
print(t.to_string(index=False))

522647
522710
     1
